In [3]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup, NavigableString, Tag
import numpy as np
import pandas as pd
from urllib.request import urlopen
import folium
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
from IPython.display import Image 
from IPython.core.display import HTML 
from pandas.io.json import json_normalize

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki = requests.get(url)

wiki_raw = pd.read_html(wiki.content, header = 0)[0]
df = wiki_raw[wiki_raw.Neighbourhood != 'Not assigned']
df.reset_index(inplace = True)
df.head()

,index,Postal Code,Borough,Neighbourhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [16]:
lat_lng_coords = None

df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.head()

df = df.join(df_geo.set_index('Postal Code'), on='Postal Code')
df.head()

,index,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,2,M3A,North York,Parkwoods,43.753259,-79.329656
1,3,M4A,North York,Victoria Village,43.725882,-79.315572
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [17]:
CLIENT_ID = 'TKEJPNG2XCZAXYJKT3TWTRONOVXJJZAWNMQX5FQ55KOXCHRO' # your Foursquare ID
CLIENT_SECRET = 'FY0YLTQWSDDDPOMBE1U25IW2HMLOKXTWC1WLAL1VNQ4KAHKW' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print( latitude, longitude)

Your credentails:
CLIENT_ID: TKEJPNG2XCZAXYJKT3TWTRONOVXJJZAWNMQX5FQ55KOXCHRO
CLIENT_SECRET:FY0YLTQWSDDDPOMBE1U25IW2HMLOKXTWC1WLAL1VNQ4KAHKW
43.6534817 -79.3839347


In [19]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        ).add_to(map_Toronto)  
    
map_Toronto

In [20]:
import json
from pandas.io.json import json_normalize
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    #initiate a new tuple of venues
    venues_list=[]
    #loop inside the 
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # get the info from foursquare
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return to the tuple of venue name, lat, lng, category
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood Name', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
venues = getNearbyVenues(names=df['Neighbourhood'],
                         latitudes=df['Latitude'],
                         longitudes=df['Longitude'])